In [1]:
%%bash
export version=`python --version |awk '{print $2}' |awk -F"." '{print $1$2}'`

echo $version

if [ $version == '36' ] || [ $version == '37' ]; then
    echo 'Starting installation...'
    pip3 install pyspark==2.4.8 wget==3.2 pyspark2pmml==0.5.1 > install.log 2> install.log
    if [ $? == 0 ]; then
        echo 'Please <<RESTART YOUR KERNEL>> (Kernel->Restart Kernel and Clear All Outputs)'
    else
        echo 'Installation failed, please check log:'
        cat install.log
    fi
elif [ $version == '38' ] || [ $version == '39' ]; then
    pip3 install pyspark==3.1.2 wget==3.2 pyspark2pmml==0.5.1 > install.log 2> install.log
    if [ $? == 0 ]; then
        echo 'Please <<RESTART YOUR KERNEL>> (Kernel->Restart Kernel and Clear All Outputs)'
    else
        echo 'Installation failed, please check log:'
        cat install.log
    fi
else
    echo 'Currently only python 3.6, 3.7 , 3.8 and 3.9 are supported, in case you need a different version please open an issue at https://github.com/IBM/claimed/issues'
    exit -1
fi

37
Starting installation...
Please <<RESTART YOUR KERNEL>> (Kernel->Restart Kernel and Clear All Outputs)


In [2]:
# @param data_dir temporal data storage for local execution
# @param data_csv csv path and file name (default: data.csv)
# @param data_parquet path and parquet file name (default: data.parquet)
# @param master url of master (default: local mode)

In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark2pmml import PMMLBuilder
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler
import pandas as pd
import logging
import shutil
import site
import sys
import wget
import re
import os
import shutil
import glob

In [4]:
if sys.version[0:3] == '3.9':
    url = ('https://github.com/jpmml/jpmml-sparkml/releases/download/1.7.2/'
           'jpmml-sparkml-executable-1.7.2.jar')
    wget.download(url)
    shutil.copy('jpmml-sparkml-executable-1.7.2.jar',
                site.getsitepackages()[0] + '/pyspark/jars/')
elif sys.version[0:3] == '3.8':
    url = ('https://github.com/jpmml/jpmml-sparkml/releases/download/1.7.2/'
           'jpmml-sparkml-executable-1.7.2.jar')
    wget.download(url)
    shutil.copy('jpmml-sparkml-executable-1.7.2.jar',
                site.getsitepackages()[0] + '/pyspark/jars/')
elif sys.version[0:3] == '3.7':
    url = ('https://github.com/jpmml/jpmml-sparkml/releases/download/1.5.12/'
           'jpmml-sparkml-executable-1.5.12.jar')
    wget.download(url)
elif sys.version[0:3] == '3.6':
    url = ('https://github.com/jpmml/jpmml-sparkml/releases/download/1.5.12/'
           'jpmml-sparkml-executable-1.5.12.jar')
    wget.download(url)
else:
    raise Exception('Currently only python 3.6 , 3.7, 3,8 and 3.9 is supported, in case '
                    'you need a different version please open an issue at '
                    'https://github.com/IBM/claimed/issues')

In [10]:
data_parquet = os.environ.get('data_parquet', 'data.parquet')        # input file name (parquet)
data1_csv = os.environ.get('data1_csv', 'data1.csv')                 # output file name (csv)
data2_csv = os.environ.get('data2_csv', 'data2.csv')                 # output file name (csv)
model_target = os.environ.get('model_target', "model.xml")           # model output file name
master = os.environ.get('master', "local[*]")                        # URL to Spark master
data_dir = os.environ.get('data_dir', '../../data/')                 # temporary directory for data
input_columns = os.environ.get('input_columns', '["x", "y", "z"]')   # input columns to consider

In [11]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

for parameter in parameters:
    logging.warning('Parameter: ' + parameter)
    exec(parameter)

In [12]:
conf = SparkConf().setMaster(master)
#if sys.version[0:3] == '3.6' or sys.version[0:3] == '3.7':
conf.set("spark.jars", 'jpmml-sparkml-executable-1.5.12.jar')

sc = SparkContext.getOrCreate(conf)
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession

In [15]:
data_parquet = 'data.parquet'
data2_csv = 'data2.csv'

In [16]:
#Read in the parquet file you created as part of Task 3.
df = spark.read.parquet(data_dir + data_parquet)

In [17]:
#register a corresponding query table
df.createOrReplaceTempView('df')

In [18]:
#cast parquet int columns to doubles
from pyspark.sql.types import DoubleType
df = df.withColumn("x", df.x.cast(DoubleType()))
df = df.withColumn("y", df.y.cast(DoubleType()))
df = df.withColumn("z", df.z.cast(DoubleType()))

In [19]:
#Convert the parquet file to CSV format (first check if data1.csv already exists from previous runs).
if not os.path.exists(data_dir + data2_csv):
    df.write.option("header", True).option("delimiter",",").csv(data_dir + data2_csv)

In [20]:
#Load the CSV file into a dataframe
df1 = spark.read.csv(data_dir + data2_csv)

In [21]:
#register a corresponding query table
df1.createOrReplaceTempView('df1')

In [22]:
df2 = df1.withColumnRenamed("_c0", "x")
df2 = df2.withColumnRenamed("_c1", "y")
df2 = df2.withColumnRenamed("_c2", "z")
df2 = df2.withColumnRenamed("_c3", "source")
df2 = df2.withColumnRenamed("_c4", "class")

In [23]:
#register a corresponding query table
df2.createOrReplaceTempView('df2')

In [24]:
#cast csv int columns to doubles
from pyspark.sql.types import DoubleType
df2 = df2.withColumn("x", df2.x.cast(DoubleType()))
df2 = df2.withColumn("y", df2.y.cast(DoubleType()))
df2 = df2.withColumn("z", df2.z.cast(DoubleType()))

In [25]:
#Create a 80-20 training and test split with seed=1
splits = df2.randomSplit([0.8, 0.2], 1)
df2_train = splits[0]
df2_test = splits[1]

In [26]:
indexer = StringIndexer(inputCol="class", outputCol="label")

vectorAssembler = VectorAssembler(inputCols=eval(input_columns),
                                  outputCol="features")

normalizer = MinMaxScaler(inputCol="features", outputCol="features_norm")

In [27]:
#pmmlBuilder = PMMLBuilder(sc, df1_train, model)
#pmmlBuilder.buildFile(data_dir + model_target)

In [28]:
#Train a Random Forest model with different hyperparameters listed below and report the best performing hyperparameter combinations.
#Hyper parameters:
#  - number of trees : {10, 20}
#  - maximum depth : {5, 7} 
#  - use random seed = 1 wherever needed

bestAccuracy = 0.0
bestNTrees = 10
bestMaxDepth = 5

for curNTrees in [10, 20]:
    for curMaxDepth in [5, 7]:
        rf = RandomForestClassifier(numTrees=curNTrees, maxDepth=curMaxDepth, seed=1)
        pipeline = Pipeline(stages=[indexer, vectorAssembler, normalizer, rf])
        model = pipeline.fit(df2_train)
        prediction = model.transform(df2_train)
        binEval = MulticlassClassificationEvaluator(). \
            setMetricName("accuracy"). \
            setPredictionCol("prediction"). \
            setLabelCol("label")
        curAccuracy = binEval.evaluate(prediction)
        print("accuracy = {} with params {}, {}".format(curAccuracy, curNTrees, curMaxDepth))
        if curAccuracy > bestAccuracy:
            bestNTrees = curNTrees
            bestMaxDepth = curMaxDepth
            bestAccuracy = curAccuracy

print("Best accuracy = {} with params {}, {}".format(bestAccuracy, bestNTrees, bestMaxDepth))

[Stage 7:>                                                          (0 + 7) / 7]23/06/28 15:09:35 ERROR executor.Executor: Exception in task 6.0 in stage 7.0 (TID 31)
org.apache.spark.SparkException: Failed to execute user defined function($anonfun$4: (struct<x:double,y:double,z:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$class.foreach(Iterator.scala:891)
	at scala.collection.AbstractIterator.f

Py4JJavaError: An error occurred while calling o91.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 7.0 failed 1 times, most recent failure: Lost task 6.0 in stage 7.0 (TID 31, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$4: (struct<x:double,y:double,z:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.It